### Гипотезы
1. Убрать ненужные слова по типу: расскажи, докажи, из, ...

    Есть два пути: выбрать опорные слова из определений и оставить только их или собрать служебные и второстепенные слова и выкинуть служебные
2. На каком языке лучше (хотим русский)
3. Дообучить русскую нейронку (недавно вышла Т-pro)

## качаю нужные библиотеки

In [ ]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 64.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0fd2689e7a8f948320105d2df5b9717396c4afb011a9b61614a9e02f743d381e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.2 MB/s eta 0:00:00


In [ ]:
!pip install 'pdfminer.six[image]'

## всякие импорты

In [ ]:
import re # регулярки
import nltk # для стоп-слов
from nltk.tokenize import sent_tokenize # для токенизации
from nltk.tokenize import RegexpTokenizer # разбиваем на слова
from nltk.corpus import stopwords # убираем стоп слова
import pymorphy2 # для лемантизации
from collections import Counter # подсчёт частых
from gensim.models.phrases import Phrases, Phraser # дли биграмм
from scipy import spatial

from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from pdfminer.high_level import extract_text # pdf в текст

from google.colab import drive # подключение к диску

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
stopwords_ru = stopwords.words('russian')
tokenizer = RegexpTokenizer('\w+')
morph = pymorphy2.MorphAnalyzer()

## начинаем работу с текстами

### книга

In [ ]:
# лекции за 2023 год + материалы итмо + книга хопкрофта на русском
all_texts = []

for i in range(1, 69):
    t = extract_text(f"/content/drive/MyDrive/texts/{i}.pdf")
    # глазами зацепилась за не оч хорошие переносы, поэтому фикс
    t = t.replace('-\n', '').replace('\n\n', '\n')
    all_texts.append(t)

## код

### какие-то мои функции

In [ ]:
# подсчёт метрики k
def my_metrics(user, db, clean_dict=None, trash_dict=None):
    user = make_clean(user)
    db = make_clean(db)

    user = get_words_only(user)
    db = get_words_only(db)

    if clean_dict is not None:
        user = [w for w in user if w in my_dict]
        db = [w for w in db if w in my_dict]

    if trash_dict is not None:
        user = [w for w in user if w not in trash_dict]
        db = [w for w in db if w not in trash_dict]

    print(user)
    print(db)

    in_both = list(set(user) & set(db))

    return len(in_both) / (len(db) + epsilon)

In [ ]:
# берём только слова из предложений
def get_words_only(sentences_tokenize):
    words = [item for sent in sentences_tokenize for item in sent]

    return words

In [ ]:
def make_clean(text):
    # чистим текст
    text = text.lower()
    text = re.sub('\n|\t|\r', ' ', text)
    # преобразуем в предложения
    sentences = sent_tokenize(text)
    # бъём предложения на слова
    sentences_tokenize = [tokenizer.tokenize(item) for item in sentences]
    # убираем стоп слова
    sentences_tokenize = [[item for item in sent if (item not in stopwords_ru and re.match('^[а-я]*$', item))] for sent in sentences_tokenize]
    # лемантизируем слова
    sentences_tokenize = [[morph.normal_forms(item)[0] for item in sent] for sent in sentences_tokenize]

    return sentences_tokenize

In [ ]:
# словарь популярных слов
def get_dict_words(sentences_tokenize):
    words = [item for sent in  sentences_tokenize for item in sent]
    word_dict = Counter(words)
    common = word_dict.most_common()

    return common

In [ ]:
# полезные слова = часто встречаем их
def get_useful(common):
    my_dict = {}

    for i, j in common:
        if i != 'ε' and len(i) > 2 and re.match('^[а-я]*$', i):
            my_dict[i] = j

    return my_dict

### смотрим на текст

In [ ]:
epsilon = 0.001

all_sentences = []
all_words = []

my_dict = {}

# мусорные конструкции
trash_dict = ['че', 'доказать', 'рассказать', 'ваш', 'привести', 'сделать', 'описать', 'дать', 'что']

In [ ]:
for text in all_texts:
    t = make_clean(text)
    if len(t) > 0:
        all_sentences = all_sentences + t

    # частые слова
    common = get_dict_words(t)
    all_words += [i for (i, j) in common]
    # «полезные» слова
    useful_dict = get_useful(common)

    new_dict = dict(list(my_dict.items()) + list(useful_dict.items()))
    my_dict = new_dict.copy()

### предобработка перед doc2vec

In [ ]:
vocab_index = [TaggedDocument(text,[k]) for k, text in enumerate(all_sentences)]

model = Doc2Vec(vector_size=300, window=1, min_count=3, workers=6, epochs=20)

model.build_vocab(vocab_index)
model.train(vocab_index, total_examples=model.corpus_count, epochs=20)

In [ ]:
s1 = model.infer_vector('дка'.split())
s2 = model.infer_vector('регулярные языки и их свойства'.split())

cos_distance = spatial.distance.cosine(s1, s2)

cos_distance

0.9651259338896611

In [ ]:
similar_vec = model.docvecs.most_similar([s2], topn=10)

similar_sent = [" ".join(vocab_index[top[0]].words) for top in similar_vec]

<ipython-input-65-29078a78222d>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_vec = model.docvecs.most_similar([s2], topn=10)


### предобработка перед w2v

In [ ]:
bigram = Phrases(all_sentences)
bigram_transformer = Phraser(bigram)

# генератор текстов с биграммами
def bigram_generator():
    for text in all_sentences:
        yield bigram_transformer[[word for word in text]]

In [ ]:
trigram = Phrases(bigram_generator())
trigram_transformer = Phraser(trigram)

def trigram_generator():
    for text in all_sentences:
        yield trigram_transformer[bigram_transformer[[word for word in text]]]

In [ ]:
ok = [i for i in trigram_generator()]

In [ ]:
model = Word2Vec(vector_size=300, window=10, min_count=3, workers=4)
model.build_vocab(ok)

In [ ]:
model.corpus_count

21753

In [ ]:
model.train(ok, total_examples=model.corpus_count, epochs=500)

(56711352, 67266500)

In [ ]:
model.wv.most_similar('дка')

[('нка', 0.47808364033699036),
 ('автомат', 0.4368232488632202),
 ('состояние', 0.3628830313682556),
 ('процесс_допуск', 0.2999037206172943),
 ('нда', 0.2974991500377655),
 ('множество', 0.2651441693305969),
 ('так_быть', 0.2553767263889313),
 ('эквивалентный_дка', 0.24655848741531372),
 ('заключительный_состояние', 0.24137888848781586),
 ('начальный_состояние', 0.2403261363506317)]

In [ ]:
# сохраню потом
model.save('wv_model')

In [ ]:
trigram_transformer.save('wv_trigramm')

# подвал

In [ ]:
# чистим текст из пдфки
test = make_clean(all_texts[0])

In [ ]:
# словарь с частыми словами
common = get_dict_words(test)

In [ ]:
# словарь с полезными терминами
my_dict = get_useful(common)

In [ ]:
s1 = 'че такое эти ваши производные антимирова'

s2 = 'Как можно вычислить частичные производные Антимирова?'

In [ ]:
my_metrics(s1, s2, trash_dict=trash_dict)

['такой', 'производный', 'антимиров']
['вычислить', 'частичный', 'производный', 'антимиров']


0.4998750312421894

и тут для меня дошло, что, если расписывать вопросы в БЗ, то деление на длину слов вопроса из БЗ (коих будет много) не будет явно и точно отражать близость, но относительно такой штуки можно смотреть динамику